In [3]:
import argparse
import sys

sys.path.insert(0,'..')
from utils.digraph import DiGraph
from utils.esitimater import Estimater

class InfluenceNetwork(DiGraph):
    '''
    Inheritance from Digraph
    '''
    def __init__(self):
        DiGraph.__init__(self)
        self.spec = {'nodes':-1, 'edges':-1}
    
    def load_from_file(self, filed):
        '''
        load from the file
        '''
        lines = filed.readlines()
        self.spec['nodes'] = int(lines[0][0])
        self.spec['edges'] = int(lines[0][1])
        for line in lines[1:]:
            data = line.split()
            if len(data) == 3:
                self.add_weighted_edge((int(data[0]), int(data[1])), float(data[2]))
                # print((int(data[0]), int(data[1])), float(data[2]))

def loadseeds(filed):
    '''
    load seeds from the file
    '''
    seeds = set()
    lines = filed.readlines()
    for line in lines:
        data = line.split()
        if len(data) == 1:
            seeds.add(int(data[0]))
    return seeds

In [4]:
network = open('../network.txt')
seed_f = open('../seeds.txt')

graph = InfluenceNetwork()
graph.load_from_file(network)
seeds = loadseeds(seed_f)
print(seeds)
print(graph.edges())

network.close()
seed_f.close()

set([1, 12, 14, 15])
[set([9, 4]), set([10, 6]), set([10, 7]), set([1, 11]), set([11, 3]), set([10, 14]), set([14, 6]), set([14, 7]), set([1, 15]), set([4, 15]), set([16, 1]), set([17, 15]), set([2, 18]), set([10, 18]), set([18, 14]), set([18, 7]), set([16, 19]), set([8, 20]), set([2, 20]), set([9, 21]), set([19, 21]), set([17, 21]), set([19, 22]), set([18, 23]), set([16, 25]), set([25, 19]), set([25, 15]), set([18, 26]), set([2, 27]), set([26, 27]), set([8, 28]), set([27, 28]), set([2, 28]), set([26, 28]), set([18, 28]), set([9, 29]), set([2, 29]), set([21, 29]), set([19, 30]), set([25, 30]), set([11, 30]), set([30, 22]), set([8, 31]), set([20, 31]), set([29, 31]), set([32, 18]), set([33, 10]), set([33, 14]), set([17, 34]), set([34, 13]), set([34, 22]), set([34, 15]), set([34, 35]), set([35, 15]), set([36, 30]), set([24, 37]), set([2, 37]), set([21, 37]), set([34, 38]), set([35, 38]), set([37, 38]), set([9, 38]), set([38, 15]), set([17, 38]), set([38, 22]), set([17, 39]), set([34, 39]

In [5]:
graph[9][4]

{'weight': 0.333333}

In [4]:
graph.inverse[4]

{9: {'weight': 0.333333}, 15: {'weight': 0.333333}, 60: {'weight': 0.333333}}

In [5]:
import random
from __future__ import division

In [10]:
def ic_simulate():
    activated = set()
    next_layer = seeds
    while next_layer:
        new_layer = set()
        for node in next_layer:
            for linked_node, value  in graph[node].iteritems():
                rnd = random.random()
                if linked_node not in activated and rnd < value['weight']:
                    new_layer.add(linked_node)
        activated = set.union(activated, next_layer)
        print(activated)
        next_layer = new_layer
    return activated

def lt_simulate():
    activated = seeds
    threshold = dict()
    for node in graph.vertices():
        threshold[node] = random.random()
    changed = True
    while changed:
        changed = False
        inactive = set.difference(set(graph.vertices()), activated)
        for node in inactive:
            indicator = 0 
            for linked_node, value  in graph.inverse[node].iteritems():
                if linked_node in activated:
                    indicator += graph.inverse[node][linked_node]['weight']
            if indicator > threshold[node]:
                activated.add(node)
                changed = True
    return activated
        
        
    

In [11]:
ic_simulate()

set([1, 12, 14, 15])
set([1, 6, 7, 12, 14, 15])


{1, 6, 7, 12, 14, 15}

In [12]:
lt_simulate()

{1, 4, 6, 7, 10, 12, 14, 15}

In [13]:
[0]*19

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [9]:
import Queue as Q
import heapq


In [16]:
(6,-100)<(3,4)

False

In [25]:
import random

In [27]:
from __future__ import division
def ic_evaluate(seeds):
    cnt = 0
    for _ in range(10000):
        activated = set()
        next_layer = seeds
        while next_layer:
            new_layer = set()
            for node in next_layer:
                for linked_node, value  in graph[node].iteritems():
                    rnd = random.random()
                    if linked_node not in activated and rnd < value['weight']:
                        new_layer.add(linked_node)
            activated = set.union(activated, next_layer)
            next_layer = new_layer
        cnt += len(activated)
    return cnt/10000

In [40]:
num_k = 4

def ic_celf():
    '''
    implementation of CELF
    '''
    state_list = list()
    free_nodes = set(graph.vertices())
    cur_spread = 0
    cur_set = set()
    # init the heap, note this is a minheap
    for nodeid in free_nodes:
        new_spread = ic_evaluate(set.union(cur_set, {nodeid}))
        state_list.append((-new_spread, nodeid))
    heapq.heapify(state_list)
    inserted_node = heapq.heappop(state_list)
    cur_set.add(inserted_node[1])
    cur_spread = -inserted_node[0]
    free_nodes.remove(inserted_node[1])
    cur_max = 1
    while len(cur_set) < num_k:
        next_node = heapq.heappop(state_list)
        if next_node[0] < cur_max:
            new_spread = ic_evaluate(set.union(cur_set, {next_node[1]}))
            diff = new_spread - cur_spread
            next_node = (-diff, next_node[1])
            cur_max = next_node[0]
            heapq.heappush(state_list, next_node)
        else:
            inserted_node = heapq.heappop(state_list)
            # print(inserted_node)
            cur_set.add(inserted_node[1])
            cur_spread += -inserted_node[0]
            cur_max = 1
    return cur_set, cur_spread


In [42]:
import time
st = time.time()
print(ic_celf())
print(time.time()-st)

(-5.3655, 53)
(-3.0679000000000016, 62)
(-2.8843999999999994, 50)
(set([56, 50, 53, 62]), 22.736)
None
9.07210183144
